<a href="https://colab.research.google.com/github/Stefano97/AN2DL_Project2/blob/master/Preprocessing_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
rm "/content/drive/My Drive/startDataset" -R

In [ ]:
!git clone https://github.com/Stefano97/AN2DL_Project2

In [ ]:
! pip uninstall kaggle

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c ann-and-dl-image-segmentation

In [ ]:
!unzip ./*.zip

In [ ]:
! rm "/content/drive/My Drive/maschere" -R


In [ ]:
! cp -R "/content/ann-and-dl-image-segmentation.zip" "/content/drive/My Drive/"

In [ ]:
! ls "drive/My Drive/Segmentation_Dataset/training/masks/img"

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#%tensorflow_version 2.x
import tensorflow as tf

SEED = 1234
tf.random.set_seed(SEED) 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_img_data_gen = ImageDataGenerator(rescale=1./255,validation_split=0.3)
train_mask_data_gen = ImageDataGenerator(rescale=1./255,validation_split=0.3)

In [ ]:
import os

cwd = os.getcwd()
dataset_dir = os.path.join(cwd, 'Segmentation_Dataset')

# Batch size
bs = 4

# img shape
img_h = 256
img_w = 256

num_classes=21

# Training
# Two different generators for images and masks
# ATTENTION: here the seed is important!! We have to give the same SEED to both the generator
# to apply the same transformations/shuffling to images and corresponding masks
training_dir = os.path.join(dataset_dir, 'training')
train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, # Because we have no class subfolders in this case
                                                       shuffle=True,
                                                       interpolation='bilinear',
                                                       seed=SEED,
                                                       subset="training")  
train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         class_mode=None, # Because we have no class subfolders in this case
                                                         shuffle=True,
                                                         interpolation='bilinear',
                                                         color_mode="grayscale",
                                                         seed=SEED,
                                                         subset="training")
valid_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, # Because we have no class subfolders in this case
                                                       shuffle=True,
                                                       interpolation='bilinear',
                                                       seed=SEED,
                                                       subset="validation")  
valid_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         class_mode=None, # Because we have no class subfolders in this case
                                                         shuffle=True,
                                                         interpolation='bilinear',
                                                         color_mode="grayscale",
                                                         seed=SEED,
                                                         subset="validation")

train_gen = zip(train_img_gen, train_mask_gen)
valid_gen = zip(valid_img_gen, valid_mask_gen)

def prepare_target(x_, y_):
    y_ = tf.cast(y_, tf.int32)
    return x_, y_

train_gen = train_gen.map(prepare_target)
valid_gen = valid_gen.map(prepare_target)

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np

%matplotlib notebook


fig,ax = plt.subplots(1,2)
fig.show()

iterator = iter(train_gen)

for _ in range(1000):
    img, target = next(iterator)
    img = img[0]   # First element
    img = img * 255  # denormalize
    target = target[0]
    
    target = target.squeeze(-1)
    
    
    # Assign colors (just for visualization)
    target_img = np.zeros([256, 256, 3])

    target_img[np.where(target == 0.0)] = [0, 0, 0]
    target_img[np.where(target == 1.0)] = [255, 255, 255]


    ax[0].imshow(np.uint8(img))
    ax[1].imshow(np.uint8(target_img))


    fig.canvas.draw()
    time.sleep(1)

In [ ]:
import json
import re

training = []
for imgName in train_img_gen.filenames:
    matchObj = re.search(r'[^\\\\/]+$',imgName,re.M)    
    file_name = matchObj.group()
    training.append(file_name)
    
validation = []
for imgName in valid_img_gen.filenames:
    matchObj = re.search(r'[^\\\\/]+$',imgName,re.M)    
    file_name = matchObj.group()
    validation.append(file_name)

# dictionary with the format shown in the Evaluation tab
dataset_split = {'training': training , 'validation': validation}   
with open('dataset_split.json', 'w') as fp:
    json.dump(dataset_split, fp)


